In [23]:
import glob
import importlib
import os
import time

import numpy as np
import torch
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import random_split
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

import models.hgp_sl_py311.models_new as models

In [24]:
# Refresh model so that code is updated directly
importlib.reload(models) 

<module 'models.hgp_sl_py311.models_new' from '/home/oliver/projects/uzh/aml/models/hgp_sl_py311/models_new.py'>

In [25]:
args = {
    'device': "cpu",
    'seed': 42,
    'batch_size': 64,
    'lr': 0.0001,
    'weight_decay': 0.001,
    'epochs': 10,
    'patience': 1000,
    'nhid': 128,
    'pooling_ratio': 0.3,
    'dropout_ratio': 0.5,
    'sample_neighbor': True,
    'sparse_attention': True,
    'structure_learning': True,
    'lamb': 1.0,
}

In [26]:
torch.set_printoptions(edgeitems=10)

if torch.cuda.is_available():
    args["device"] = 'cuda'
    
torch.manual_seed(args['seed'])

dataset = TUDataset(os.path.join('../../data', "DD"), name="DD", use_node_attr=True)
args["num_classes"] = dataset.num_classes
args["num_features"] = dataset.num_features

num_test = int(len(dataset) * 0.1)
num_train = len(dataset) - num_test
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

kfold = KFold(n_splits=5, shuffle=True, random_state=args['seed'])

In [27]:
def compute_test(model, loader):
    model.eval()
    correct = 0.0
    loss_test = 0.0
    for data in loader:
        data = data.to(args["device"])
        out = model(data)
        pred = out.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        loss_test += F.nll_loss(out, data.y).item()
    return correct / len(loader.dataset), loss_test

In [28]:
def train_fold(train_idx, val_idx):
    train_subset = torch.utils.data.Subset(train_dataset, train_idx)
    val_subset = torch.utils.data.Subset(train_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=args["batch_size"], shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=args["batch_size"], shuffle=False)

    model = models.Model(args).to(args["device"])
    optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])
    min_loss = 1e10
    patience_cnt = 0
    val_loss_values = []
    val_acc_values = []
    best_epoch = 0

    t = time.time()
    model.train()
    for epoch in range(args["epochs"]):
        loss_train = 0.0
        correct = 0
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = data.to(args["device"])
            out = model(data)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            pred = out.max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
        acc_train = correct / len(train_loader.dataset)
        acc_val, loss_val = compute_test(model, val_loader)
        print('Epoch: {:04d}'.format(epoch + 1), 'loss_train: {:.6f}'.format(loss_train),
              'acc_train: {:.6f}'.format(acc_train), 'loss_val: {:.6f}'.format(loss_val),
              'acc_val: {:.6f}'.format(acc_val), 'time: {:.6f}s'.format(time.time() - t))

        val_loss_values.append(loss_val)
        val_acc_values.append(acc_val)
        torch.save(model.state_dict(), '{}.pth'.format(epoch))
        if val_loss_values[-1] < min_loss:
            min_loss = val_loss_values[-1]
            best_epoch = epoch
            patience_cnt = 0
        else:
            patience_cnt += 1

        if patience_cnt == args["patience"]:
            break

        files = glob.glob('*.pth')
        for f in files:
            epoch_nb = int(f.split('.')[0])
            if epoch_nb < best_epoch:
                os.remove(f)

    files = glob.glob('*.pth')
    for f in files:
        epoch_nb = int(f.split('.')[0])
        if epoch_nb > best_epoch:
            os.remove(f)
    print('Optimization Finished! Total time elapsed: {:.6f}'.format(time.time() - t))
    return val_acc_values[best_epoch],val_loss_values[best_epoch]

In [29]:
test_loader = DataLoader(test_dataset, batch_size=args["batch_size"], shuffle=False)
results = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(np.arange(len(train_dataset)))):
    print(f"Training fold {fold + 1}")
    val_acc, val_loss = train_fold(train_idx, val_idx)
    results.append((val_acc, val_loss))

# Now train on the full training dataset (optional if you want a final model)
# Test the model

# Output cross-validation and test results
print(f"Cross-validation results: {results}")

Training fold 1
Epoch: 0001 loss_train: 9.654626 acc_train: 0.529481 loss_val: 2.731375 acc_val: 0.643192 time: 5.744173s
Epoch: 0002 loss_train: 9.602072 acc_train: 0.568396 loss_val: 2.710957 acc_val: 0.643192 time: 11.047084s
Epoch: 0003 loss_train: 9.544126 acc_train: 0.568396 loss_val: 2.694845 acc_val: 0.643192 time: 16.417342s


KeyboardInterrupt: 